# THIS IS UNSUCCESSFUL NOTEBOOK - DON'T FOLLOW

# Quora Insincere Questions Classifications

Original: [Basic Model](https://www.kaggle.com/nitinaggarwal008/basic-model)

In [7]:
# !pip install tensorflow
# !pip install --upgrade keras
# !pip install --upgrade tensorflow
# !pip install --upgrade tensorflow-gpu

In [1]:
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Layer
from keras.optimizers import Adam
from keras import backend as K
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn import metrics

from tqdm import tqdm
import pandas as pd
import numpy as np
import string, re
import time
import math
import os

## Setup

In [15]:
embed_size = 300
max_features = 53
maxlen = 70

In [3]:
def clean(text):
    # Remove punctuation
    text = text.translate(string.punctuation)
    
    # Convert words to lower case and split
    text = text.lower()
    
    ## remove stop words
    # text = text.split()
    # stops = set(stopwords.words("english"))
    # text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    # Clean the text
    text = re.sub(r"[^A_Za-z0-9^,!>\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"\'re", "are", text)
    text = re.sub(r"\'d", "would", text)
    text = re.sub(r"\'ll", "will", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg", text)
    text = re.sub(r" b g", " bg", text)
    text = re.sub(r" u s", " american", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub("[^a-zA-Z]", " ", text)
    text = re.sub("  +", " ", text)
    
    # text = text.split()
    # stemmer = SnowballStemmer("english")
    # stemmed_words = [stemmer.stem(word) for word in text]
    # text = " ".join(stemmed_words)
    return text

Load training and test data: Convert into padded sentences for keras model inputs

In [4]:
def load_and_prec():
    df_train = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
    df_test = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
    print("Train shape:", df_train.shape)
    print("Test shape:", df_test.shape)
    
    df_train['clean_text'] = df_train['question_text'].apply(clean)
    df_test['clean_text'] = df_test['question_text'].apply(clean)
    
    # Split into train and val
    df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=21)
    
    # Fill up the missing values
    X_train = df_train['clean_text'].fillna("_##_").values
    X_test = df_test['clean_text'].fillna("_##_").values
    X_val = df_val['clean_text'].fillna("_##_").values
    
    # Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(X_train))
    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)
    X_val = tokenizer.texts_to_sequences(X_val)
    
    # Padding
    X_train = pad_sequences(X_train, maxlen=maxlen)
    X_test = pad_sequences(X_test, maxlen=maxlen)
    X_val = pad_sequences(X_val, maxlen=maxlen)
    
    # Get the target values
    y_train = df_train['target'].values
    y_val = df_val['target'].values
    
    # Shuffle data
    np.random.seed(21)
    train_idx = np.random.permutation(len(X_train))
    val_idx = np.random.permutation(len(X_val))
    
    X_train = X_train[train_idx]
    X_val = X_val[val_idx]
    y_train = y_train[train_idx]
    y_val = y_val[val_idx]
    
    return X_train, X_val, X_test, y_train, y_val, tokenizer.word_index

## Embedding setup

Source: [keras embeddings](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)

In [5]:
!unzip ../input/quora-insincere-questions-classification/embeddings.zip

Archive:  ../input/quora-insincere-questions-classification/embeddings.zip
   creating: GoogleNews-vectors-negative300/
   creating: glove.840B.300d/
   creating: paragram_300_sl999/
   creating: wiki-news-300d-1M/
  inflating: glove.840B.300d/glove.840B.300d.txt  
  inflating: GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin  
  inflating: wiki-news-300d-1M/wiki-news-300d-1M.vec  
  inflating: paragram_300_sl999/README.txt  
  inflating: paragram_300_sl999/paragram_300_sl999.txt  


In [6]:
def load_gloves(word_index):
    EMBEDDING_FILE = './glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    
    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

In [7]:
def load_fasttext(word_index):
    EMBEDDING_FILE = './wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word, * arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o) > 100)
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    
    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

In [8]:
def load_para(word_index):
    EMBEDDING_FILE = './paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word, * arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o) > 100)
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    
    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

## Define LSTM Model Architecture

In [17]:
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation='relu')(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## Define Training Functions

In [10]:
def train_pred(model, epochs=5):
    for e in range(epochs):
        model.fit(X_train, y_train, batch_size=512, epochs=1, validation_data=(X_val, y_val))
        y_val_pred = model.predict([X_val], batch_size=1024, verbose=0)
        
        best_thresh = 0.5
        best_score = 0.0
        for thresh in np.arange(0.1, 0.501, 0.01):
            thresh = np.round(thresh, 2)
            score = metrics.f1_score(y_val, (y_val_pred > thresh).astype(int))
            if score > best_score:
                best_thresh = thresh
                best_score = score
                
            print("Val F! Score: {:.4f}".format(best_score))
            
        y_test_pred = model.predict([X_test], batch_size=1024, verbose=0)
        return y_val_pred, y_test_pred, best_score

## Load the data

In [11]:
%%time
X_train, X_val, X_test, y_train, y_val, word_index = load_and_prec()

Train shape: (1306122, 3)
Test shape: (375806, 2)
CPU times: user 5min 20s, sys: 3.58 s, total: 5min 23s
Wall time: 5min 26s


## Load the embeddings

In [12]:
%%time
embedding_matrix_1 = load_gloves(word_index)
# embedding_matrix_2 = load_fasttext(word_index)
# embedding_matrix_3 = load_para(word_index)

/opt/conda/lib/python3.7/site-packages/IPython/core/magics/execution.py:1321: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code, glob, local_ns)


CPU times: user 4min 40s, sys: 13 s, total: 4min 53s
Wall time: 4min 53s


## Run the model

In [23]:
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    # x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [33]:
def train_pred(model, epochs=2):
    for e in range(epochs):
        model.fit(X_train, y_train, batch_size=2048, epochs=1, validation_data=(X_val, y_val))
        y_val_pred = model.predict([X_val], batch_size=2048, verbose=1)

        best_thresh = 0.5
        best_score = 0.0
        for thresh in np.arange(0.1, 0.501, 0.01):
            thresh = np.round(thresh, 2)
            score = metrics.f1_score(y_val, (y_val_pred > thresh).astype(int))
            if score > best_score:
                best_thresh = thresh
                best_score = score

        print("Val F1 Score: {:.4f}".format(best_score))
    y_test_pred = model.predict([X_test], batch_size=2048, verbose=1)
    return y_val_pred, y_test_pred, best_score

In [34]:
%%time
y_val_pred, y_test_pred, best_score = train_pred(model_lstm_du(embedding_matrix_1), epochs=1)

196/511 [==========>...................] - ETA: 2:08 - loss: 0.2688 - accuracy: 0.9115

InvalidArgumentError:  indices[1477,60] = 53 is not in [0, 53)
	 [[node model_5/embedding_8/embedding_lookup (defined at <ipython-input-33-a2a4dacc82ac>:3) ]] [Op:__inference_train_function_12024]

Errors may have originated from an input operation.
Input Source operations connected to node model_5/embedding_8/embedding_lookup:
 model_5/embedding_8/embedding_lookup/11690 (defined at /opt/conda/lib/python3.7/contextlib.py:112)

Function call stack:
train_function


## Choose the best threshold

In [26]:
thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(y_val, (y_val_pred > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F! score at threshold {0} is {1}".format(thresh, res))
    
threholds.sort(key=lambda x: x[1], reverse=True)
best_thresh = thresholds[0][0]
print("Best threshold:", best_thresh)

NameError: name 'y_val_pred' is not defined

## Confusion matrix of validation output

In [ ]:
metrics.confusion_matrix(y_val, y_val_pred > best_thresh)

In [ ]:
y_pred_test = (y_test_pred > best_thresh).astype(int)
df_test = pd.read_csv("", usecols=["qid"])
df_out = pd.DataFrame({"qid":df_test["qid"].values})
df_out['predictions'] = y_test_pred
df_out.to_csv("submission.csv", index=False)

# THIS IS AN ERROR, SO I JUST WANT TO SAVE THE PROGRESS ONLY